Análise exploratória e modelo de classificação Random Forest

In [353]:
# Biblioteca pandas
import pandas as pd
# Biblioteca para realizar o label encoder
from sklearn.preprocessing import LabelEncoder
# Biblioteca para dividir os dados entre treino e teste
from sklearn.model_selection import train_test_split
# Biblioteca para criar a matriz de confusão e obter a acurácia do modelo
from sklearn.metrics import confusion_matrix, accuracy_score
# Algoritmo Random Forest
from sklearn.ensemble import RandomForestClassifier

A base de dados utilizada nessa atividade é sobre a perda de clientes de telecomunicações, contendo 20 colunas com os dados dos clientes e 1 coluna com a informação sobre a retenção de clientes.

In [354]:
# Fazendo a leitura da base de dados
df = pd.read_csv('Telco-Customer-Churn.csv')

In [355]:
# Verificando as primeiras e as últimas informações da base de dados
display(df)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [356]:
# Removendo a primeira coluna "customerID", que se trata do ID único do cliente, já que essa informação não é importante para a análise
df = df.drop('customerID', axis='columns')

In [357]:
# Verificando se os tipos dos dados condizem com as informações presentes nela
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

A variável "TotalCharges" está como object na base, porém os dados desta coluna são valores decimais, 
portanto é necessário fazer a modificação do seu tipo

In [358]:
# Modificando o tipo da variável "TotalCharges" para numérico
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# Verificando se o tipo da variável foi alterado
df['TotalCharges'].dtypes

dtype('float64')

In [359]:
# Verificando se a base de dados possui algum valor NA
df.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [360]:
# A variável "TotalCharges" possui 11 linhas onde os valores são NA, dessa forma iremos substituir esses valores pela média dessa coluna
# Obtendo a média da variável
media = df['TotalCharges'].mean()
# Substituindo os valores NA pela média da variável
df['TotalCharges'] = df['TotalCharges'].fillna(media)
# Verificando se a variável teve seus valores NA substituídos
df['TotalCharges'].isnull().sum()

0

Agora, será aplicado a codificação nas colunas categóricas da base de dados, para que seja possível fazer a classificação com o algoritmo Random Forest

In [361]:
# Laço de repetição que irá percorrer todas as colunas da base de dados
for i in range(len (df.columns)):
    # Verifica se a coluna é do tipo object
    if(df.iloc[:,i].dtype == object):
        # Realiza o label encoder na coluna
        df.iloc[:,i] = LabelEncoder().fit_transform(df.iloc[:,i])

In [362]:
# Verificando se a codificação foi realizada
display(df)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,2,0,2,0,2,2,2,2,1,1,3,84.80,1990.50,0
7039,0,0,1,1,72,1,2,1,0,2,2,0,2,2,1,1,1,103.20,7362.90,0
7040,0,0,1,1,11,0,1,0,2,0,0,0,0,0,0,1,2,29.60,346.45,0
7041,1,1,1,0,4,1,2,1,0,0,0,0,0,0,0,1,3,74.40,306.60,1


In [363]:
# Seleciona os atributos da base de dados
X = df.iloc[:,0:19]
# Seleciona a classe da base de dados
y = df.iloc[:,19]

In [364]:
# Divisão da base de dados entre treinamento e teste, sendo 70% para treinamento e 30% para teste
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,
                                                                  y,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

In [365]:
# Criação e treinamento do modelo utilizando o algoritmo supervisionado Random Forest
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest.fit(X_treinamento, y_treinamento)

RandomForestClassifier()

In [366]:
# Previsões utilizando os registros de teste
predict = random_forest.predict(X_teste)
predict

array([0, 0, 0, ..., 0, 0, 0])

In [367]:
# Cria a matriz de confusão
confusion = confusion_matrix(y_teste, predict)
confusion

array([[1392,  168],
       [ 279,  274]], dtype=int64)

In [368]:
# Obtêm a acurácia obtida pelo algoritmo de Naive Bayes
accuracy = accuracy_score(y_teste, predict)
accuracy

0.7884524372929484

Portanto, a acurácia obtida pelo algoritmo Random Forest foi de 79%